***
## Import

In [1]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
# YOUR CODE HERE (OPTION) 
import datetime
# Nếu cần các thư viện khác thì bạn có thể import ở đây

import urllib.robotparser # Kiểm tra file robot.txt có được phép crawl không

In [2]:
sleep_time = 1 # Đơn vị: giây

## ParseHTML

In [29]:
# profileHeaderInfo__userName
def collect_playlist(playlist_url, playlist_file):
    with open(playlist_file,'w', encoding='utf-8') as fo:
        with open(playlist_url,'r', encoding='utf-8') as fin:
            for url_text in fin.readlines():
                print(url_text + '\n')
                time.sleep(sleep_time)
                
                req = requests.get(url_text)
                html_text = req.text
                soup = BeautifulSoup(html_text, features="lxml")
                
                title = soup.find('div',class_='soundTitle__titleHeroContainer')
                print(str(title) + "\t")
                
collect_playlist('url.csv','dataframe.tsv')

https://soundcloud.com/sieu-nhan-ruoi/sets/playlist-nhacviet-morning


None	
